In [1]:
# !pip install datasets

In [2]:
import datasets

qa = datasets.load_dataset('squad', split='validation')
qa

Found cached dataset squad (C:/Users/u042/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

In [3]:
qa[0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [4]:
qa[1]

{'id': '56be4db0acb8001400a502ed',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the NFC at Super Bowl 50?',
 'answers': {'text': ['Carolina Panthers',
   'Carolina Panthers',
   'Caroli

In [5]:
unique_contexts = []
unique_ids = []

# make list of IDs that represent only first instance of each context
for row in qa:
    if row['context'] not in unique_contexts:
        unique_contexts.append(row['context'])
        unique_ids.append(row['id'])
        
# now filter out any sample that aren't included in unique IDs
qa = qa.filter(lambda x: True if x['id'] in unique_ids else False)
qa

Loading cached processed dataset at C:\Users\u042\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-c2c2462b8b5781fc.arrow


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2067
})

In [6]:
# !pip install sentence_transformers

### Create context vectors with the retriever model

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

### Encode the context vectors

In [8]:
qa = qa.map(lambda x: {
    'encoding': model.encode(x['context']).tolist()
}, batched=True, batch_size=32)

qa

Loading cached processed dataset at C:\Users\u042\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-8e560c6ebb305938.arrow


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'encoding'],
    num_rows: 2067
})

## Create Vector Database (and index context vectors)

#### Use either Faiss (there is a whole set of articles and videos dedicated to Faiss at Pinecone. For Pinecone we need an API key from: https://app.pinecone.io.

c8cfcdd4-e091-47ca-be53-4c2c6c4bfbb4

#### Also install the client:

!pip install pinecone-client


In [9]:
# !pip install pinecone-client

In [10]:
API_KEY = "c8cfcdd4-e091-47ca-be53-4c2c6c4bfbb4"

In [11]:
import pinecone

pinecone.init(API_KEY, environment='us-east1-gcp')

In [12]:
# pinecone.create_index('qa-index', dimension=len(model.encode('hello world').tolist()))

In [13]:
index = pinecone.Index('qa-index')

In [14]:
from tqdm.auto import tqdm

upserts = [(v['id'], v['encoding']) for v in qa]
for i in tqdm(range(0, len(upserts), 50)):
    i_end = i + 50
    if i_end > len(upserts):
        i_end = len(upserts)
    index.upsert(vectors=upserts[i:i_end])

  0%|          | 0/42 [00:00<?, ?it/s]

## QA Inference

In [15]:
# query = "Which NFL team represented the AFC at Super Bowl 50?"
query = "Do NFL teams only care about playing at the Super Bowl?"
xq = model.encode([query]).tolist()

In [16]:
xc = index.query(xq, top_k=5)
# xc

In [17]:
xc['matches']

[{'id': '56bebcbe3aeaaa14008c9325',
  'score': 0.589542806,
  'sparseValues': {},
  'values': []},
 {'id': '56bebe873aeaaa14008c933f',
  'score': 0.577623963,
  'sparseValues': {},
  'values': []},
 {'id': '56be53b8acb8001400a50314',
  'score': 0.572862089,
  'sparseValues': {},
  'values': []},
 {'id': '56be5523acb8001400a5032c',
  'score': 0.563994706,
  'sparseValues': {},
  'values': []},
 {'id': '56be54bdacb8001400a50322',
  'score': 0.542263329,
  'sparseValues': {},
  'values': []}]

In [18]:
ids = [x['id'] for x in xc['matches']]
# ids
contexts = qa.filter(lambda x: True if x['id'] in ids else False)

Loading cached processed dataset at C:\Users\u042\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-cc5e057d3d4342c9.arrow


In [19]:
from transformers import pipeline

model_name = 'yjernite/bart_eli5'
nlp = pipeline(tokenizer=model_name, model=model_name,
              task='text2text-generation') # seq2seq

In [20]:
for context in contexts['context']:
    print(nlp(
        f"question: {query} context: {context}",
        num_beams=4,
        do_sample=True,
        temperature=1.5,
        max_length=64
    ))
    break

[{'generated_text': " The best way to think about the Super Bowl is like this: There are two types of people that play football. Some of them play for a team and some play for their own team. Some of these people are good, some of them aren't. Some of those people are the same kind of people that"}]
